In [1]:

from flask import Flask,Response
from flask import request,jsonify
from flask_sqlalchemy import SQLAlchemy
from flask import current_app
import pandas as pd
import subprocess
import json
import os
import sys
current_directory = os.getcwd()
base_directory = os.path.join(current_directory, '../../common/backend/')
base_directory = os.path.normpath(base_directory)  
sys.path.insert(1, base_directory)
from model import * 
from util import *
# %matplotlib widget
import seaborn as sns
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time 
from sqlalchemy.orm import aliased
from sqlalchemy import asc
from functools import lru_cache
import json
import numpy as np
import pymysql
import configparser
from sqlalchemy import func, distinct

config_path = os.path.join(current_directory, "../../config/qaas-web.conf")
config = configparser.ConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read(config_path)
lore_database_uri = config['web']['SQLALCHEMY_DATABASE_URI_LORE']
db = SQLAlchemy()
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = lore_database_uri
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db.init_app(app)
def with_app_context(func):
    def wrapper(*args, **kwargs):
        with app.app_context():
            return func(*args, **kwargs)
    return wrapper

In [2]:
# @with_app_context
# def lore_metrics_table_test():
#     src_loop_id = 450656 
#     vendor = "icc" 
#     version = "17.0.1"
#     src_loop = db.session.query(SrcLoop).filter_by(table_id = src_loop_id).one()
#     metrics = src_loop.loops[0].lore_loop_measures[0].lore_loop_measure_metrics
#     for metric in metrics:
#         key = metric.metric_name
#         value = metric.metric_value
#         # print(key)
#         try:
#             float_value = float(value)
#             # print(key, value)
#             print(key)

#         except:            
#             # print(key, value)
#             pass
# lore_metrics_table_test()

In [3]:
# @with_app_context
# def souce_metrics_table_test():
#     src_loop_id = 450656 
#     vendor = "icc" 
#     version = "17.0.1"
#     src_loop = db.session.query(SrcLoop).filter_by(table_id = src_loop_id).one()
#     metrics = src_loop.source.source_metrics
#     print(type(metrics))
#     for metric in metrics:
#         key = metric.metric_name
#         value = metric.metric_value
#         # print(key)
#         try:
#             float_value = float(value)
#             # print(key, value)
#             print(key)

#         except:            
#             # print(key, value)
#             pass
# souce_metrics_table_test()

In [4]:
#data cleanning
file_path = os.path.join(current_directory, 'compiler_prediction_model_data.csv')
if os.path.exists(file_path):
    df = pd.read_csv(file_path, sep=',')



In [5]:
df[['original_loop_id','base_median', 'rating','is_best_compiler']].head(20)

,original_loop_id,base_median,rating,is_best_compiler
0,1,768.0,0.050000,0.0
1,1,736.0,0.062500,0.0
2,1,724.0,0.083333,0.0
3,1,752.0,0.058824,0.0
4,1,728.0,0.076923,0.0
5,1,812.0,0.045455,0.0
6,1,896.0,0.043478,0.0
7,1,928.0,0.041667,0.0
8,1,728.0,0.076923,0.0
9,1,728.0,0.076923,0.0


In [6]:
df.head()

,original_loop_id,mutated_loop_id,mutation_number,compiler_vendor_version,restructured,compiled,exe_passed,tiny,pluto,n_mutation_attempts,...,o3_pin_vector_st_cnt,o3_pin_fp_arith_inst_cnt,o3_pin_fp_sse_inst_cnt,o3_pin_fp_avx_inst_cnt,o3_pin_int_sse_inst_cnt,o3_pin_int_avx_inst_cnt,o3_pin_vector_inst_exclude_mem_cnt,o3_pin_vector_other_inst_cnt,is_best_compiler,rating
0,1,9,1,clang_3.4.2,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000
1,1,10,1,clang_3.6.2,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062500
2,1,11,1,clang_4.0.0,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333
3,1,12,1,gcc_4.7.4,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058824
4,1,13,1,gcc_4.8.5,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923


In [7]:
df.columns

Index(['original_loop_id', 'mutated_loop_id', 'mutation_number',
       'compiler_vendor_version', 'restructured', 'compiled', 'exe_passed',
       'tiny', 'pluto', 'n_mutation_attempts',
       ...
       'o3_pin_vector_st_cnt', 'o3_pin_fp_arith_inst_cnt',
       'o3_pin_fp_sse_inst_cnt', 'o3_pin_fp_avx_inst_cnt',
       'o3_pin_int_sse_inst_cnt', 'o3_pin_int_avx_inst_cnt',
       'o3_pin_vector_inst_exclude_mem_cnt', 'o3_pin_vector_other_inst_cnt',
       'is_best_compiler', 'rating'],
      dtype='object', length=406)

In [19]:
#when is gcc a winner
gcc_best_compilers = df[(df['compiler_vendor_version'].str.contains("gcc")) & (df['is_best_compiler'] == 1)]
gcc_best_compilers

,original_loop_id,mutated_loop_id,mutation_number,compiler_vendor_version,restructured,compiled,exe_passed,tiny,pluto,n_mutation_attempts,...,o3_pin_vector_st_cnt,o3_pin_fp_arith_inst_cnt,o3_pin_fp_sse_inst_cnt,o3_pin_fp_avx_inst_cnt,o3_pin_int_sse_inst_cnt,o3_pin_int_avx_inst_cnt,o3_pin_vector_inst_exclude_mem_cnt,o3_pin_vector_other_inst_cnt,is_best_compiler,rating
2212,2237,2264,3,gcc_4.7.4,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2260,2301,2328,3,gcc_4.7.4,1.0,1.0,1.0,0.0,1.0,16.0,...,437.0,443.0,0.0,439.0,0.0,1.0,440.0,1311.0,1.0,1.0
2366,2429,2440,1,gcc_6.2.0,1.0,1.0,0.0,0.0,0.0,4.0,...,32766.0,131070.0,0.0,32767.0,0.0,0.0,32767.0,98298.0,1.0,1.0
2400,2453,2482,3,gcc_6.2.0,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2430,2485,2520,4,gcc_4.7.4,1.0,1.0,1.0,0.0,1.0,32.0,...,478.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553378,565332,565336,1,gcc_6.2.0,1.0,1.0,0.0,0.0,1.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
553427,565380,565393,1,gcc_6.2.0,1.0,1.0,1.0,0.0,1.0,16.0,...,0.0,256368.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
553595,565564,565585,2,gcc_6.2.0,1.0,1.0,1.0,0.0,1.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
553781,565772,565799,3,gcc_4.7.4,1.0,1.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [9]:
df.shape

(554218, 406)

In [ ]:
#predict rating for each loop then see which compiler it predicted has highest rating to know what is the best compielr
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
import numpy as np
from sklearn.preprocessing import LabelEncoder

data = df

# encode categorical variables
categorical_cols = ['compiler_vendor_version']
encoder = OneHotEncoder(sparse=False)
encoded_categorical_columns = encoder.fit_transform(data[categorical_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical_columns, columns=encoder.get_feature_names_out(categorical_cols))

numeric_data = data.drop(categorical_cols + ['base_median', 'is_best_compiler','rating', 'novec_median', 'sse_median', 'avx_median', 'avx2_median', 'o3_median'], axis=1)
processed_data = pd.concat([numeric_data, encoded_categorical_df], axis=1)

X = processed_data
y = data['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
#random forest regression
from sklearn.ensemble import RandomForestRegressor 
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the regressor on the training set
regressor.fit(X_train, y_train)

# Predict ratings on the test set
y_pred = regressor.predict(X_test)

# Evaluate the regressor
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')
# feature importance
feature_importances = regressor.feature_importances_
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
print(importance_df.sort_values(by='Importance', ascending=False))

Mean Squared Error (MSE): 0.05587823398742418
R-squared (R2): 0.4129718516626063
                                       Feature  Importance
301                   o3_L1D_PEND_MISS_PENDING    0.062313
117              novec_CPU_CLK_UNHALTED_THREAD    0.055972
159                sse_CPU_CLK_UNHALTED_THREAD    0.054565
201                avx_CPU_CLK_UNHALTED_THREAD    0.046007
66                                      o3_min    0.040403
..                                         ...         ...
251                         avx2_L2_RQSTS_MISS    0.000000
252               avx2_L2_DEMAND_RQSTS_WB_MISS    0.000000
253      avx2_MEM_LOAD_UOPS_RETIRED_L1_MISS_PS    0.000000
95   base_DTLB_STORE_MISSES_MISS_CAUSES_A_WALK    0.000000
203               avx_UNC_IMC_DRAM_DATA_WRITES    0.000000

[406 rows x 2 columns]


In [11]:
#see what is the predictied best compielr
X_test_with_predictions = X_test.copy()
X_test_with_predictions['predicted_rating'] = y_pred

X_test_with_predictions['predict_best_compiler'] = 0
for _, group in X_test_with_predictions.groupby('original_loop_id'):
    # row indwx with the min base_median
    best_compiler_idx = group['predicted_rating'].idxmax()
    X_test_with_predictions.at[best_compiler_idx, 'predict_best_compiler'] = 1

X_test_with_predictions[['original_loop_id', 'predicted_rating', 'compiler_vendor_version_clang_3.6.2',
       'compiler_vendor_version_clang_4.0.0',
       'compiler_vendor_version_gcc_4.7.4',
       'compiler_vendor_version_gcc_4.8.5',
       'compiler_vendor_version_gcc_6.2.0',
       'compiler_vendor_version_icc_15.0.6',
       'compiler_vendor_version_icc_17.0.1','predict_best_compiler']]

,original_loop_id,predicted_rating,compiler_vendor_version_clang_3.6.2,compiler_vendor_version_clang_4.0.0,compiler_vendor_version_gcc_4.7.4,compiler_vendor_version_gcc_4.8.5,compiler_vendor_version_gcc_6.2.0,compiler_vendor_version_icc_15.0.6,compiler_vendor_version_icc_17.0.1,predict_best_compiler
31,33,0.234446,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
15,1,0.097924,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
26,33,0.401181,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
17,1,0.595711,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,1,0.130891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,1,0.201693,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
19,1,0.182864,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0


In [12]:
X_test_with_predictions[['original_loop_id','mutation_number', 'predicted_rating','predict_best_compiler']]

,original_loop_id,mutation_number,predicted_rating,predict_best_compiler
31,33,3,0.234446,0
15,1,2,0.097924,0
26,33,1,0.401181,1
17,1,3,0.595711,1
8,1,2,0.130891,0
9,1,2,0.201693,0
19,1,3,0.182864,0


In [13]:
# random forest
# from sklearn.metrics import accuracy_score, classification_report

# model = RandomForestClassifier(n_estimators=100, random_state=42)

# # train
# model.fit(X_train, y_train)
# # predictions
# predictions = model.predict(X_test)
# print(predictions)
# print(y)
# # lose function
# accuracy = accuracy_score(y_test, predictions)
# report = classification_report(y_test, predictions)
# print(f'Accuracy: {accuracy}')
# print(report)
# feature importance
# feature_importances = model.feature_importances_
# importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
# print(importance_df.sort_values(by='Importance', ascending=False))


In [14]:
#check what data we hava in lore dynamic metrics table
@with_app_context
def value_test():
    src_loop_id = 1
 
    vendor = "icc" 
    version = "17.0.1"
    src_loop = db.session.query(SrcLoop).filter_by(table_id = src_loop_id).one()
    compiler = src_loop.loops[0].compiler_option.compiler
    print(src_loop.mutation_number, src_loop.execution.application.workload, src_loop.execution.application.version, src_loop.execution.application.program, compiler.vendor, compiler.version, src_loop.file)
value_test()

0 spec2000 v1.3 253.perlbmk clang 3.4.2 regcomp.c_regclass_line2073_loop.c.0.c
